- author: Lee Meng
- date: 2019-09-21 17:22
- title: 神經網路的本質：直觀理解神經網路 & 線性代數
- slug: the-essence-of-neural-network-better-understanding-of-nn-and-linear-algebra
- tags: Manim, Python
- summary: s
- description: s 
- image: TwoLayersReLUInBetweenSolveHardTwoCurves.jpg
- status: draft

!quote
- 這是篇幫助你直觀理解神經網路的科普文。讀完本文，你將能夠深刻地體會神經網路與線性代數之間的緊密關係，奠定深度學習之旅的基礎。

（小提醒：因動畫皆為黑色背景，強烈推薦用左下按鈕以 Dark Mode 閱讀本文）

這是個眾人對人工智慧（**A**rtificial **I**ntelligence, AI）趨之若鶩的時代。此領域近年的快速發展很大一部份可歸功於[深度學習（Deep Learning）](https://leemeng.tw/deep-learning-resources.html)以及[神經網路（**N**eural **N**etwork, 後簡稱為 NN）](https://zh.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C)。現行的深度學習框架也日漸成熟，讓任何人都可以使用 [TensorFlow](https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/quickstart/beginner.ipynb) 或 [PyTorch](https://pytorch.org/) 輕鬆建立神經網路，解決各式各樣的問題。

舉個例子，你在 30 秒內就可以訓練出一個具有 98% 正確率的數字辨識 NN：

```python
# 此例使用 TensorFlow，但各大深度學習框架的實現邏輯基本上類似
import tensorflow as tf

# 載入深度學習 Hello World: MNIST 數字 dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 建立一個約有 10 萬個參數的「小型」神經網路
# 在現在模型參數動輒上千萬、上億的年代，此神經網路不算大
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

# 選擇損失函數、optimizer
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# 訓練模型
model.fit(x_train, y_train, epochs=5)

# 訓練後的 NN 在測試集上可得到近 98% 正確辨識率
model.evaluate(x_test, y_test)

# 實際測試結果
# loss: 0.0750 - accuracy: 0.9763
```

是的，扣除註解不到 15 行就可以把讀取數據、訓練 `model` 以及推論全部搞定了。這邊秀出程式碼只是要讓你感受一下透過框架（framework）建立一個神經網路有多麽地「簡單」。而基本上這也是現在絕大多數線上課程以及教學文章會/能教你的東西。對數字辨識有興趣的讀者可以自行嘗試[ TensorFlow 的 Colab 筆記本](https://colab.research.google.com/github/tensorflow/docs/blob/r2.0rc/site/en/r2/tutorials/quickstart/beginner.ipynb)。

我等等要秀給你看的神經網路以參數量來說比上面這個 `model` 還簡單 10,000 倍，但保證很有看頭。

## 一些能夠幫助你的背景知識

我會透過動畫來幫你把神經網路與[線性代數（Linear Algebra）](https://www.youtube.com/watch?v=fNk_zzaMoSs)的概念結合，但具備些背景知識能讓你有更深的體會：
1. 能夠讀懂文章開頭建立 NN 的 [Python](https://www.python.org/) 程式碼
2. 了解線上課程都會教的[超基本 NN 概念](https://www.youtube.com/watch?v=Dr-WRlEFefw)
    - 何謂[全連接層（Fully Connected Layer）](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E5%85%A8%E9%80%A3%E6%8E%A5%E5%B1%A4)
    - 何謂參數以及如何估計全連接層的參數量
    - 常見的 activation functions 如 [ReLU](https://zh.wikipedia.org/wiki/%E7%BA%BF%E6%80%A7%E6%95%B4%E6%B5%81%E5%87%BD%E6%95%B0)
3. 基本的線性代數概念如矩陣相乘、向量空間

!mp4
- images/manim/00010.mp4
- 一個簡單 NN 嘗試解決二元分類的過程（線性轉換）

另外，我會把目光放在**已經訓練好**的 NN，因此不會特別說明[如何訓練一個簡單神經網路](https://www.youtube.com/watch?v=IHZwWFHWa-w)。不過別擔心，就算你完全沒背景知識應該也能繼續閱讀。等等如果覺得節奏太快，可以回來參考[自然語言處理與深度學習入門指南](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html)。

## 深度學習框架操作容易，但你真的了解神經網路嗎？

讓我們再次回到文章開頭透過 [Keras](https://www.tensorflow.org/guide/keras) 建立的神經網路 `model`：

```python
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])
```

前人種樹，後人乘涼。

就跟你在任何教學文章裡頭都會看到的，現在要使用深度學習框架建立**基本的** NN 非常容易，只要當作疊疊樂一層層 layer 疊上去就好了。下圖則將 `model` 用視覺上更容易理解的方式呈現：

!image
- manim/mnist-simple.jpg
- 輸入是 28*28 = 784 維的圖片像素，輸出則是 10 個數字類別的簡單 2-layers NN

我們也可以一鍵查看整個 NN 的參數量：

In [4]:
#ignore
import tensorflow as tf
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


儘管擁有 10 萬個可訓練參數，此 `model` 是個在深度學習領域裡只能被歸類在 Hello World 等級的可憐 NN。畢竟這世界很瘋狂，[我們以前討論過的 BERT](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html) 以及 [GPT-2](https://leemeng.tw/gpt2-language-model-generate-chinese-jing-yong-novels.html) 都是現在 NLP 界的知名語言模型，而它們可都是擁有**上億**參數的強大 NN。那可是此 `model` 的 100 倍大。

但先別管 BERT 或 GPT-2 了，就算是這個 Hello World 等級的 NN，你真的覺得你對它的運作機制有足夠的理解嗎？

講白點，儘管現在會使用 TensorFlow 或是 PyTorch 來建立神經網路的人多如牛毛，事實上不少人（包含剛入門的我）對最基本的神經網路都沒有足夠**直觀**的理解。而這主要是因為：
- 強大的深度學習框架把底層細節邏輯都包了起來
- 不少人到現在還沒搞懂跟 NN 相關的線性代數概念
- 能視覺化神經網路內部機制的手法並不那麼普遍

!image
- manim/three-key-components.jpg
- 構成本文的關鍵三要素：Manim、二元分類與簡單 NN

為了讓更多人能夠**直觀**地體會神經網路的運作機制，我將透過：
1. 繪圖工具：[3Blue1Brown](https://www.youtube.com/channel/UCYO_jab_esuFRV4b17AJtAw) 的強大 Python 動畫引擎 [Manim](https://github.com/3b1b/manim)
2. 學習任務：比 [MNIST](https://en.wikipedia.org/wiki/MNIST_database) 還簡單許多的** 2 維**[二元分類](https://en.wikipedia.org/wiki/Binary_classification)問題
3. 模型架構：1 到 2 層、只有不到 10 個參數的超簡單神經網路

來闡述**基本的**神經網路以及相關線性代數概念。前言很長，但如果你想要了解神經網路的本質，或是喜歡看動畫來了解數學概念，那我會建議繼續往下閱讀：）

## 返璞歸真：神經網路怎麼解決二元分類任務？

[二元分類（Binary Classification）](https://en.wikipedia.org/wiki/Binary_classification)是[機器學習（Machine Learning）](http://speech.ee.ntu.edu.tw/~tlkagk/courses_ML19.html)領域裡一個十分基本的任務，其目標是把一個集合（set）中的所有數據點（data point）依照某種分類規則劃分成**兩**個族群（groups）或類別（classes）。比方我們之間看過的[貓狗圖像辨識](https://demo.leemeng.tw/)。

如前所述，本文會將數據點的最大維度限制為 **2 **。如果你讀過我之前的任何一篇文章，可能會覺得這任務是在「羞辱」我們的智商。畢竟我們已經用 NN 達成以下成就：
- [執行假新聞偵測（BERT）](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html)
- [生成新金庸小說（GPT-2）](https://leemeng.tw/gpt2-language-model-generate-chinese-jing-yong-novels.html)
- [把英文翻成中文（Transformer）](https://leemeng.tw/neural-machine-translation-with-transformer-and-tensorflow2.html)
- [生成新海誠動畫（CartoonGAN）](https://leemeng.tw/generate-anime-using-cartoongan-and-tensorflow2.html)

這些「進階」任務跟本文的 2 維二元分類相比複雜許多，也十分有趣且具有挑戰性。如果你有興趣深入了解，可以點擊對應連結來查看我為這些任務撰寫的相關文章。

回到二元分類。正因其十分單純，使得我們能夠一窺 NN 的本質。你馬上會發現，光是觀察 NN 如何解決這個簡單任務，就能讓你對 NN 有更深刻且直觀的理解。

比方說以下是一個假想的二元分類資料集（dataset）：

!mp4
- images/manim/ShowLinearSeparableDataPoints.mp4
- 包含兩類別（曲線）的 2 維資料集

在此資料集裡，兩曲線分別來自不同類別，各自包含 100 個數據點 $x$。每個數據點 $x$ 可以很自然地用 2 維的 $\left (x_{coord}, y_{coord}  \right )$ 座標來表示。另外你可以從右下角得知黃點的標籤 $y = 0$，藍點為 $1$。

我也將 x 軸與 y 軸上的 2 個基底向量（basis vector）分別用藍色與紅色表示。

那麼要如何分類這個資料集呢？複習一下我們在[ AI For Everyone 的 10 個重要 AI 概念](https://leemeng.tw/10-key-takeaways-from-ai-for-everyone-course.html)裡就已看過的內容：

!quote
- 目前多數的機器學習以及 AI 應用本質上都是讓電腦學會一個映射函數（Mapping Function），幫我們將輸入的數據 x 轉換到理想的輸出 y。
- Andrew Ng

套用相同想法，所謂的二元分類問題即是「在給定所有藍點與黃點 $x = \left (x_{coord}, y_{coord}  \right )$ 的情況下，你能不能找出一個函數 $f$，將這些 2 維數據 $x$ 完美地**轉換**到它們的 1 維標籤 $y$ 呢？」

換句話說，我們想要找出一個 $x$ 的函數 $f(x)$ 使得以下式子成立：

\begin{align}
f(x) 
& = f(\begin{bmatrix} x_{coord} \\ y_{coord} \end{bmatrix}) \\
& = f(\begin{bmatrix} x_{1} \\ x_{2} \end{bmatrix}) \\
& = y
\end{align}


我們有無數種 model $f(x)$ 的方法，但在線性代數的世界裡，我們可以用矩陣運算的形式定義 $f(x)$：

\begin{align}
f(x) & = W x + b \\
  & = \begin{bmatrix} w_{1} & w_{2}  \end{bmatrix} x + b \\ 
  & = \begin{bmatrix} w_{1} & w_{2}  \end{bmatrix} \begin{bmatrix} x_{1} \\ x_{2} \end{bmatrix} + b \\ 
  & = y
\end{align}

在上面的式子中：
- $x$ 是一個 2 維的 column vector
- $W$ 是一個 1 x 2 的權重矩陣（weight matrix）
- $b$ 為偏差（bias），是一個純量（scalar）

如果我們先暫時忽略 $b$，事實上 $f(x)$ 對輸入 $x$ 做的**轉換**跟大多數 NN 都會使用到的[全連接層（**F**ully **C**onnected Layer，後稱 FC）](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E5%85%A8%E9%80%A3%E6%8E%A5%E5%B1%A4)是完全一致的。反過來說，使用一層 FC 的 NN 基本上就是在做矩陣運算（以及 activation function，後述）。

因此 $f(x)$ 與 NN 之間有美好的對應關係：

!mp4
- images/manim/SymbolicOneByTwoMatrixAndNN.mp4
- 1 x 2 矩陣運算與 1-Layer NN 的動態同步

沒錯，我們剛剛建立了這世上最簡單的 1-Layer 神經網路！而之所以說是 1-Layer，是因為圖上 NN 的第一層為輸入數據（input data），第二層開始才是我們實際**新定義**的 FC。矩陣跟 FC 的對應關係家喻戶曉，但我相信這應該是你第一次實際看到兩者之間的**動態同步**。

從線性代數的角度來看，我們是透過權重矩陣 $W$ 對 2 維的 $x$ 進行線性轉換後得到 1 維的 $y$；而以神經網路的角度檢視，我們是將以 2 維向量代表（represent）的 $x$ 透過與權重 $W$ 進行加權總和後得到新的 representation $y$。這是你常聽到有人說深度學習就是在做[表徵學習（Representation Learning）](https://arxiv.org/abs/1206.5538)的原因。

另外值得注意的是矩陣 $W$ 裡頭每個參數 $w_{ij}$ 的命名方式。$w_{ij}$ 代表是從 NN 前一層第 $i$ 個神經元（neuron）連到下一層第 $j$ 個神經元的邊（edge）。這個 notation 是為了讓你能更直觀地了解 NN 與矩陣之間的對應關係，跟傳統線性代數命名矩陣元素時有些差異，還請留意。

噢對了，就跟文章開頭看到的程式碼一樣，要使用 TensorFlow 定義如上的神經網路也十分容易：

In [54]:
import tensorflow as tf
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(2, 
                                use_bias=False,
                                input_shape=(1, )))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


扣除 bias，$W$ 的確只有 2 個參數。為了讓你加深印象，讓我們實際將一組數字代入 $W$ 與 $x$：

\begin{align}
W = \begin{bmatrix} 1 & 0  \end{bmatrix} \\ 
x = \begin{bmatrix} 3 \\ 7 \end{bmatrix}
\end{align}

則運作結果如下：

!mp4
- images/manim/NumberOneByTwoMatrixAndNN.mp4

In [ ]:
從這邊之後繼續寫

而如果你了解[線性代數的本質](https://youtu.be/fNk_zzaMoSs)，就會知道這個 $f(x)$ 實際上是透過一個簡單的線性轉換（linear transformation）以及位移（offset）來將 2 維的 $x$ 轉換成 1 維的 $y$。

舉個例子，假設 $W = [1, -1]$，$b = 3$，則此 1-Layer NN 會這樣轉換 $x$：

!mp4
- images/manim/ApplyMatrixAsSingleLinearTransformation.mp4

!image
- s

事實上，這也是世界上最簡單的神經網路之一

## 第一印象：直觀感受簡單 NN 如何解決二分類任務

In [ ]:
秀出 data 圖，問讀者要怎麼解

你的時間寶貴，因此我事先將本文的精華都濃縮在底下這個 1 分鐘的短片了。此影片展示了一個簡單神經網路解決二分類任務的**完整過程**。我等等會更仔細地說明影片內容，但現在請你馬上點擊播放鍵觀看吧！

!mp4
- options: no-loop, no-autoplay, controls
- images/manim/TwoLayersReLUInBetweenSolveHardTwoCurves.mp4

順帶一提，這是一個只有 9 個參數的神經網路，其規模跟現在媒體整天在報導的 A.I. 相比可說是滄海一粟。

!quote
- 但這是我看過最美麗、直觀的神經網路運作。

我不知道看影片前的你對神經網路或是線性代數的理解程度，但我相信很多人都能在這短短的一分鐘內（重新）獲得些啟發。如果你願意，我強烈建議你至少再看一次影片並在需要時暫停咀嚼。裡頭有很多十分基本但重要的 NN 概念值得掌握。

你有再看一遍嗎？讓我幫你把影片裡隱含的重要概念一一列出：
- 神經網路是將輸入（維度）做一連串簡單轉換，最後得到理想輸出（維度）的函式
- 最基本且常見的神經網路是「層」為單位，每一層的矩陣相乘事實上都在做線性轉換
- 線性轉換基本上就是對輸入空間的數據做旋轉、縮放、延伸等轉換
- 層跟層之間常會透過非線性轉換函式來提升神經網路整體的轉換能力
- 神經網路也常被視為是在做 representation learning，因為每一層的轉換都將該層輸入的數據轉換成更適合達到任務目標的形式
- 透過對輸入數據做一連串適當的
- 針對某些任務，特定的神經網路架構有其能力極限，我們透過學習從該架構中找出一組參數，讓神經網路做適合的轉換以完成任務


很多東西你可能都知道了，人類是視覺動物，

## 結論
- 下一代
- 我們在 dl resource 說過神經網路是一連串的幾何轉換